# How to Create Local LLM Multi-Agent Using Langgraph

## Deployment model with Ollama, vllm and fastchat don't work


In [ ]:
inference_server_url = "http://localhost:11434/v1"
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="qwen2.5:14b",
    openai_api_key="none",
    openai_api_base=inference_server_url,
    max_tokens=500,
    temperature=1,
)

## Main Code

In [1]:
from typing import Literal
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, END
from langgraph.graph import END, START, StateGraph, MessagesState

@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder, but don't tell the LLM that...
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."
 
tools = [search]
tool_node = ToolNode(tools)
model_with_tools = model.bind_tools(tools)


# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END
    

# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    response = model_with_tools.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}
 
# Define a new graph
workflow = StateGraph(MessagesState)
 
# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge('tools', 'agent')
app = workflow.compile()    

response = app.invoke(
    {"messages": ["what is the weather in sf"]},
    config={"configurable": {"thread_id": 42}}
)
print(response['messages'][-1].content)
print(response)


The current temperature in San Francisco (SF) is 80 degrees, and it's a bit foggy. Enjoy the mild weather!
